In [20]:
import pandas as pd
import numpy as np
import itertools
#from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
import scipy.stats
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import seaborn as sns
from matplotlib import cm, pyplot as plt
from scipy import stats as st
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from torch import nn, optim
import torch.nn.functional as F
import torch
import brevitas.nn as qnn
from brevitas.quant import Int8WeightPerTensorFixedPoint as WeightQuant
from brevitas.quant import Int8ActPerTensorFixedPoint as ActQuant
from brevitas.quant import Int8BiasPerTensorFixedPointInternalScaling as BiasQuant
from brevitas.export import PyXIRManager
from brevitas.export import FINNManager
from preProcessing import *


## ORDER BOOK DATA
Getting the Data
Let’s download the data samples from LOBSTER. This service provides Google, Apple, Amazon, Intel, Microsoft assets as an examples with 3 levels as market depth (1, 5, 10 levels).

In [7]:

RANDOM_SEED = 42
## reading csv files and format the data adding column names
## set Asset and level 
asset = 'AAPL'
level = 1     # Can only be 1, 5 and 10

data1 = loadDataOrderBook('../../Data/{0}_2012-06-21_34200000_57600000_orderbook_{1}.csv', asset, level)
data2 = loadDataMessage('../../Data/{0}_2012-06-21_34200000_57600000_message_{1}.csv', asset, level)

# displaying input files
print("ORDER BOOK")
print("data1.shape = ", data1.shape)
print(data1.head())

print("MESSAGES")
print("data2.shape = ", data1.shape)
print(data2.head())

data1.columns=["ask", "volume_ask", "bid", "volume_bid"]
data2.columns=["Time", "Type", "OrderID", "Size", "Price", "Direction"]



ORDER BOOK
data1.shape =  (118496, 4)
   5859400  200  5853300  18
0  5859100   18  5853300  18
1  5859200   18  5853300  18
2  5859300  100  5853300  18
3  5859300  100  5853600  18
4  5859300  100  5857300  20
MESSAGES
data2.shape =  (118496, 4)
   34200.004241176  1  16113575  18  5853300  1.1
0     34200.025552  1  16120456  18  5859100   -1
1     34200.201743  3  16120456  18  5859100   -1
2     34200.201781  3  16120480  18  5859200   -1
3     34200.205573  1  16167159  18  5853600    1
4     34200.271740  1   3647217  20  5857300    1


## Merging of the order Book and the Messages

In [9]:
#Change the output format to be between 0.1 & 0.9 instead of -1 & 1
result_high=0.9
result_low=1-result_high

# using merge function by setting how='outer'
result = data1.merge(data2, left_index=True, right_index=True, how='left')
result['Direction'].replace({-1 : result_low, 1 : result_high}, inplace=True)

cols = ["ask", "volume_ask", "bid", "volume_bid", "Type", "Size", "Price", "Direction"]
result = result[cols]

print("RESULTS")  
# displaying result
print("Shape of the result matrix is = ",result.shape)
print(result.head())

print("number of rows =",result.shape[0])
print("number of columns =",result.shape[1])
print("Number of inputs for each categories")
#result.Type_1.value_counts()/result.shape[0]

RESULTS
Shape of the result matrix is =  (118496, 8)
       ask  volume_ask      bid  volume_bid  Type  Size    Price  Direction
0  5859100          18  5853300          18     1    18  5859100        0.1
1  5859200          18  5853300          18     3    18  5859100        0.1
2  5859300         100  5853300          18     3    18  5859200        0.1
3  5859300         100  5853600          18     1    18  5853600        0.9
4  5859300         100  5857300          20     1    20  5857300        0.9
number of rows = 118496
number of columns = 8
Number of inputs for each categories


## Preparing the Network Inputs

In [10]:
#Creating the 7 Inputs for the DNN 
X = result[["ask", "volume_ask", "bid", "volume_bid", "Type", "Size", "Price"]]
print("Shape of the matrix X is:\t",X.shape)

#Creating the 1 output for the DNN to train the network with results 
Y = result[["Direction"]]
print("Shape of the matrix Y is:\t",Y.shape,"\n")

##pre-processing
print("INPUT VALUES TO FEED THE CNN")
print(X.head(1))
print("OUTPUT VALUES FOR TRAINING")
print(Y.head(1))

#Normalise the INPUT value to improve the learning
X=preprocess(1)(X)



Shape of the matrix X is:	 (118496, 7)
Shape of the matrix Y is:	 (118496, 1) 

INPUT VALUES TO FEED THE CNN
       ask  volume_ask      bid  volume_bid  Type  Size    Price
0  5859100          18  5853300          18     1    18  5859100
OUTPUT VALUES FOR TRAINING
   Direction
0        0.1


### SPLIT DATA FOR TRAINING AND TESTING

In [11]:
#Split the data between trianing 80% and test 20% no shuffling as it is a time series
#0.2 = 20% of the data for test
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
X_train_np, X_test_np, Y_train_np, Y_test_np = train_test_split(X, Y, test_size=0.2, shuffle = False, stratify = None)
print("Shape of the matrix X_train_np is:\t",X_train_np.shape)
print("Shape of the matrix Y_train_np is:\t",Y_train_np.shape)
print("Shape of the matrix X_test_np is:\t",X_test_np.shape,"\n")
print("Shape of the matrix Y_test_np is:\t",Y_test_np.shape,"\n")

#checking the X_test_np data
print("INPUT",X_test_np.head(1))

#checking the Y_test_np data
print("OUTPUT",Y_test_np.head(1))

print('Convert Numpy array to Torch\n')
#_T for Torch arrays
X_train_T = torch.from_numpy(X_train_np.to_numpy()).float()

print("Dimension of input tensor:", X_train_T.dim())
print("Input tensor Size:\n",X_train_T.size())


#remove a dimension
Y_train_T = torch.squeeze(torch.from_numpy(Y_train_np.to_numpy()).float())
X_test_T = torch.from_numpy(X_test_np.to_numpy()).float()
Y_test_T = torch.squeeze(torch.from_numpy(Y_test_np.to_numpy()).float())

print("Dimension of input tensor:", Y_train_T.dim())
print("Input tensor Size:\n",Y_train_T.size())
print("Shape of the matrix X_train_T is:\t",X_train_T.shape)
print("Shape of the matrix X_test_T is:\t",X_test_T.shape,"\n")
print("Shape of the matrix Y_train_T is:\t",Y_train_T.shape)
print("Shape of the matrix Y_test_T is:\t",Y_test_T.shape,"\n")

Shape of the matrix X_train_np is:	 (94796, 7)
Shape of the matrix Y_train_np is:	 (94796, 1)
Shape of the matrix X_test_np is:	 (23700, 7) 

Shape of the matrix Y_test_np is:	 (23700, 1) 

INPUT             ask  volume_ask       bid  volume_bid      Type      Size  \
94796 -1.204252    0.098682 -1.210559   -0.032607 -0.998848  0.934232   

          Price  
94796 -1.228414  
OUTPUT        Direction
94796        0.9
Convert Numpy array to Torch

Dimension of input tensor: 2
Input tensor Size:
 torch.Size([94796, 7])
Dimension of input tensor: 1
Input tensor Size:
 torch.Size([94796])
Shape of the matrix X_train_T is:	 torch.Size([94796, 7])
Shape of the matrix X_test_T is:	 torch.Size([23700, 7]) 

Shape of the matrix Y_train_T is:	 torch.Size([94796])
Shape of the matrix Y_test_T is:	 torch.Size([23700]) 



### Building of the Pytorch Model

In [49]:
#base network
class Net(nn.Module):
  def __init__(self, n_features):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(n_features, 7)
    self.fc2 = nn.Linear(7, 20)
    self.fc3 = nn.Linear(20, 1)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    return torch.sigmoid(self.fc3(x))*(result_high-result_low)+result_low

net = Net(X_train_T.shape[1])
#print(X_train_T.shape[1])
#print("NN structure is: ",net)
#print(net.parameters)

#augmented network
class Net1(nn.Module):
  def __init__(self, n_features):
    super(Net1, self).__init__()
    self.fc1 = nn.Linear(n_features, 7)
    self.fc2 = nn.Linear(7, 20)
    self.fc4 = nn.Linear(20, 20)
    self.fc5 = nn.Linear(20, 20)
    self.fc3 = nn.Linear(20, 1)
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = F.relu(self.fc4(x))
    #x = F.relu(self.fc5(x))
    return (torch.sigmoid(self.fc3(x))*(result_high-result_low)+result_low)

net1 = Net1(X_train_T.shape[1])
#print(X_train_T.shape[1])
#print("NN structure is: ",net1)
#print(net1.parameters)


## Quantized the Network using Brevitas

In [50]:
q_bit_width=3

#quantized network
class QuantWeightNet(nn.Module):
    def __init__(self, n_features):
        super(QuantWeightNet, self).__init__()
        self.fc1   = qnn.QuantLinear(n_features, 7, bias=True, weight_bit_width=q_bit_width)
        self.relu1 = qnn.QuantReLU()
        self.fc2   = qnn.QuantLinear(7, 20, bias=True, weight_bit_width=q_bit_width)
        self.relu2 = qnn.QuantReLU()
        self.fc4   = qnn.QuantLinear(20, 20, bias=True, weight_bit_width=q_bit_width)
        self.relu3 = qnn.QuantReLU()
        self.fc5   = qnn.QuantLinear(20, 20, bias=True, weight_bit_width=q_bit_width)
        self.relu4 = qnn.QuantReLU()
        self.fc3   = qnn.QuantLinear(7, 1, bias=False, weight_bit_width=q_bit_width)
        

    def forward(self, x):
        #x = self.relu1(self.fc1(x))
        #x = self.relu2(self.fc2(x))
        #x = self.relu3(self.fc4(x))
        #x = self.relu4(self.fc5(x))
        #return (torch.sigmoid(self.fc3(x))*(result_high-result_low)+result_low)

        return ((self.fc3(x))*(result_high-result_low)+result_low)


net_q= QuantWeightNet(X_train_T.shape[1])
#print(X_train_T.shape[1])
#print("NN structure is: ",net_q)
#print(net_q.parameters)    
    


# Setting up the networks

In [51]:
#Creates a criterion that measures the Binary Cross Entropy between the target and the input probabilities
criterion = nn.BCELoss(weight=None, size_average=None, reduce=None, reduction='mean')

#optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(net.parameters(), lr=0.001)
optimizer1 = optim.Adam(net1.parameters(), lr=0.0003)
optimizer_q = optim.Adam(net_q.parameters(), lr=0.0003)



device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#transfer data to the GPU
X_train = X_train_T.to(device)
Y_train = Y_train_T.to(device)
X_test = X_test_T.to(device)
Y_test = Y_test_T.to(device)

criterion = criterion.to(device)

net = net.to(device)
net1 = net1.to(device)
net_q = net_q.to(device)

device
#print(list(net.parameters()))
#print(optimizer.param_groups)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

#print(optimizer1.param_groups)
#print(optimizer_q.param_groups)

torch.Size([94796, 7])
torch.Size([23700, 7])
torch.Size([94796])
torch.Size([23700])


# Saving network model and export as Onnx model

In [52]:
MODEL_PATH = 'model1.pt'
torch.save(net1, MODEL_PATH)
#net1 = torch.load(MODEL_PATH)
print("Done saving net")

MODEL_PATH = 'model_q.pt'
torch.save(net_q.state_dict(), MODEL_PATH)
#net_q = torch.load(MODEL_PATH)
print("Done saving Q net")


inp=torch.empty((1,1,1,7))
inp=torch.randn(1,1,1,7)
#inp = torch.tensor((1,1,1,7))

#print(inp)
print(inp.shape)


#FINNManager.export(net_q, input_shape=(1, 1, 1, 7), export_path='finn_qnet.onnx')

#from brevitas.export import export_brevitas_onnx

#export_brevitas_onnx(net_q, input_t=inp, export_path='finn_qnet.onnx')

from brevitas.export import export_finn_onnx

export_finn_onnx(net_q, input_t=inp, export_path='finn_qnet.onnx')



Done saving net
Done saving Q net
tensor([[[[ 0.2431, -0.4371,  0.8412,  0.1157, -0.1811,  2.9313, -0.3533]]]])
torch.Size([1, 1, 1, 7])


/Users/bowenpyk/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1365: UserWarning: Positional args are being deprecated, use kwargs instead. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


AttributeError: 'NoneType' object has no attribute 'shape'

### Forcasting

In [18]:
for epoch in range(1000):
      optimizer1.zero_grad() #reset Gradient descent
      optimizer_q.zero_grad()      
      Y_pred = net1(X_train)
      Y_pred_q = net_q(X_train)
      Y_pred = torch.squeeze(Y_pred)
      Y_pred_q = torch.squeeze(Y_pred_q)
      criterion=nn.BCELoss()
      train_loss = criterion(Y_pred, Y_train)
      train_loss_q = criterion(Y_pred_q, Y_train)
      train_loss.backward() #propagate the error currently making     
      optimizer1.step()      #optimise 
      train_loss_q.backward() #propagate the error currently making     
      optimizer_q.step()

      if epoch % 100==0:
        train_acc = calculate_accuracy(Y_train, Y_pred,result_high,result_low)
        train_acc_q = calculate_accuracy(Y_train, Y_pred,result_high,result_low)
        Y_test_pred = net1(X_test)
        Y_test_pred = torch.squeeze(Y_test_pred)
        test_loss = criterion(Y_test_pred, Y_test)
        test_acc = calculate_accuracy(Y_test, Y_test_pred,result_high,result_low)
        print(
  f'''epoch {epoch}
  Train set - loss: {round_tensor(train_loss)}, accuracy: {round_tensor(train_acc)}
  Test  set - loss: {round_tensor(test_loss)}, accuracy: {round_tensor(test_acc)}
  ''')
        Y_test_pred_q = net_q(X_test)
        Y_test_pred_q = torch.squeeze(Y_test_pred_q)
        test_loss_q = criterion(Y_test_pred_q, Y_test)
        test_acc_q = calculate_accuracy(Y_test, Y_test_pred_q,result_high,result_low)
        print(
  f'''epoch {epoch}
  QTrain set - loss: {round_tensor(train_loss_q)}, accuracy: {round_tensor(train_acc_q)}
  QTest  set - loss: {round_tensor(test_loss_q)}, accuracy: {round_tensor(test_acc_q)}
  ''')


epoch 0
  Train set - loss: 0.69204, accuracy: 0.52938
  Test  set - loss: 0.6958, accuracy: 0.457
  
epoch 0
  QTrain set - loss: 0.69241, accuracy: 0.52938
  QTest  set - loss: 0.69497, accuracy: 0.45772
  
epoch 100
  Train set - loss: 0.68978, accuracy: 0.52129
  Test  set - loss: 0.69405, accuracy: 0.44464
  
epoch 100
  QTrain set - loss: 0.69068, accuracy: 0.52129
  QTest  set - loss: 0.6979, accuracy: 0.45772
  
epoch 200
  Train set - loss: 0.68659, accuracy: 0.47085
  Test  set - loss: 0.68881, accuracy: 0.27156
  
epoch 200
  QTrain set - loss: 0.68842, accuracy: 0.47085
  QTest  set - loss: 0.69685, accuracy: 0.39325
  
epoch 300
  Train set - loss: 0.68121, accuracy: 0.45888
  Test  set - loss: 0.68355, accuracy: 0.23738
  
epoch 300
  QTrain set - loss: 0.68594, accuracy: 0.45888
  QTest  set - loss: 0.6928, accuracy: 0.30519
  
epoch 400
  Train set - loss: 0.67177, accuracy: 0.42214
  Test  set - loss: 0.6787, accuracy: 0.2681
  
epoch 400
  QTrain set - loss: 0.68284, 